> ## 2020.01.23 목요일

# 3. dog cat  augmentatation

- https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/
- https://www.kaggle.com/c/dogs-vs-cats/data
- 학습 데이터로 1,000장의 고양이 사진과 1,000장의 강아지 사진을 사용 (kaggle  25,000자)
- 검증 데이터로는 각각 400장 사용

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
import tensorflow as tf
import numpy as np
from numpy  import expand_dims
import cv2
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
np.random.seed(15)

Using TensorFlow backend.


In [2]:
img = load_img('cat.jpg') 
x = img_to_array(img)
print(x.shape)   # w,h,c 인직 확인

(612, 571, 3)


In [3]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음(이미지 원본을 사용)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        'smallcatdog/train', 
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = validation_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [5]:
# steps_per_epoch는 한 세대마다 몇 번 생성기로부터 데이터를 얻을지를 나타내는 값
# 한 세대마다 사용되는 학습데이터의 수는 steps_per_epoch * batch_size
        
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,    # 2000/16     한번에 125개씩 생성
        epochs=5,  #50정도 돌려야 accuracy가 80정도 나옴
        validation_data=validation_generator,
        validation_steps=800 // batch_size)     # 800/16   한번에 50개씩 생성

Epoch 1/5
125/125 [==============================] - 22s 172ms/step - loss: 0.7565 - accuracy: 0.5365 - val_loss: 1.8213 - val_accuracy: 0.5025
Epoch 2/5
125/125 [==============================] - 21s 166ms/step - loss: 0.6717 - accuracy: 0.6115 - val_loss: 0.5766 - val_accuracy: 0.6450
Epoch 3/5
125/125 [==============================] - 21s 168ms/step - loss: 0.6248 - accuracy: 0.6730 - val_loss: 0.5409 - val_accuracy: 0.6562
Epoch 4/5
125/125 [==============================] - 22s 176ms/step - loss: 0.6085 - accuracy: 0.6910 - val_loss: 0.4357 - val_accuracy: 0.6612
Epoch 5/5
125/125 [==============================] - 21s 167ms/step - loss: 0.5822 - accuracy: 0.6995 - val_loss: 0.9222 - val_accuracy: 0.7013


In [6]:
# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator( test_generator, steps = 800/16)
                                                  #전체수/배치size
print(scores[1])

-- Evaluate --
0.7012500166893005


In [7]:
# augmentation 없이  학습
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255 )
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        'smallcatdog/train',  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=5, # 50
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

model.save("smallcatdog.h5")  #save함수 쓰면 나중에 복구할때 유용함
scores = model.evaluate_generator( test_generator,       steps = 5)
print(scores[1])

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/5
125/125 [==============================] - 19s 152ms/step - loss: 0.7154 - accuracy: 0.5340 - val_loss: 0.6649 - val_accuracy: 0.5900
Epoch 2/5
125/125 [==============================] - 19s 148ms/step - loss: 0.6538 - accuracy: 0.6250 - val_loss: 0.6388 - val_accuracy: 0.6562
Epoch 3/5
125/125 [==============================] - 19s 149ms/step - loss: 0.5985 - accuracy: 0.6740 - val_loss: 0.6576 - val_accuracy: 0.6375
Epoch 4/5
125/125 [==============================] - 19s 149ms/step - loss: 0.5476 - accuracy: 0.7240 - val_loss: 0.6127 - val_accuracy: 0.7262
Epoch 5/5
125/125 [==============================] - 19s 149ms/step - loss: 0.5136 - accuracy: 0.7540 - val_loss: 0.4750 - val_accuracy: 0.6750
0.6875


In [8]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)       

## pumpkin, tomato, watermelon 구분하기

In [10]:
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'clean-dataset/clean-dataset/train', 
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='categorical') 

validation_generator = validation_datagen.flow_from_directory(
        'clean-dataset/clean-dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        'clean-dataset/clean-dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3)) #3개를 비교해야하므로
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', #categorical_crossentropy로 변경해주어야 3개비교
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(train_generator,
        steps_per_epoch=600//batch_size,    
        epochs=50,
        validation_data=validation_generator,
        validation_steps=150//batch_size)     

# 모델 평가하기
print("-- Evaluate --")
scores= model.evaluate_generator(test_generator,steps=5) 
print(scores[1])


Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Epoch 1/50
37/37 [==============================] - 5s 144ms/step - loss: 0.9306 - accuracy: 0.5394 - val_loss: 0.6250 - val_accuracy: 0.6875
Epoch 2/50
37/37 [==============================] - 5s 139ms/step - loss: 0.6801 - accuracy: 0.7277 - val_loss: 0.4764 - val_accuracy: 0.7910
Epoch 3/50
37/37 [==============================] - 5s 136ms/step - loss: 0.5402 - accuracy: 0.8099 - val_loss: 0.3555 - val_accuracy: 0.7761
Epoch 4/50
37/37 [==============================] - 5s 134ms/step - loss: 0.5298 - accuracy: 0.7877 - val_loss: 0.5359 - val_accuracy: 0.8358
Epoch 5/50
37/37 [==============================] - 5s 137ms/step - loss: 0.4304 - accuracy: 0.8345 - val_loss: 0.2414 - val_accuracy: 0.8284
Epoch 6/50
37/37 [==============================] - 5s 133ms/step - loss: 0.4276 - accuracy: 0.8559 - val_loss: 0.2805 - val_accuracy: 0.8358
Epoch 7/50
37/37 [=======

Found 800 images belonging to 2 classes.


NameError: name 'model2' is not defined

# 4. imagenet에서 검색해서 다운하기

In [9]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import PIL.Image
import urllib

In [13]:
page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n04194289") #ship synset  wnetid
soup = BeautifulSoup(page.content, 'html.parser')
str_soup=str(soup)
split_urls=str_soup.split('\r\n')
print(len(split_urls))

1262


In [14]:
bikes_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n02834778")#bicycle synset
bikes_soup = BeautifulSoup(bikes_page.content, 'html.parser')
bikes_str_soup=str(bikes_soup)
bikes_split_urls=bikes_str_soup.split('\r\n')
print(len(bikes_split_urls))

1345


In [15]:
def url_downalod(urls, path, prefix) :    
    idx = 0
    for url in urls :
        try:
            resp = urllib.request.urlopen(url)
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            if ( len(image.shape)) == 3 :
                print(url)
                idx += 1
                save_path = path + '/' + prefix + str(idx)+'.jpg'
                cv2.imwrite(save_path,image)
        except :
            None


In [ ]:
url_downalod(split_urls, 'imagenet/aa', 'ship') # aa 폴더에 ship1 부터 저장

http://static.flickr.com/85/233578996_8f74463083.jpg
http://farm2.static.flickr.com/1019/534041866_3aa182311c.jpg
http://static.flickr.com/88/231940102_7beb5ecf3e.jpg
http://farm4.static.flickr.com/3138/2724596903_d74dd64821.jpg
http://static.flickr.com/174/453920264_9406e9972c.jpg
http://farm1.static.flickr.com/147/423542865_c091691a6b.jpg
http://farm3.static.flickr.com/2269/2348294071_94fc609f4a.jpg
http://static.flickr.com/1016/1443997081_b5eb875055.jpg
http://www.fondear.org/infonautic/Barco/Barco_Navegando/Contenedores_Peligro/sea_container_ship2.jpg
http://static.flickr.com/198/491737075_0100196a8f.jpg
http://farm4.static.flickr.com/3132/2640561254_b114ac2788.jpg
http://farm1.static.flickr.com/12/14188200_7c98ba44cc.jpg
http://farm4.static.flickr.com/3226/2350689291_dbcebb71ee.jpg
http://farm4.static.flickr.com/3282/2576278846_63f669f036.jpg
http://farm4.static.flickr.com/3171/2640549706_a942ef34dc.jpg
http://farm4.static.flickr.com/3174/2596564567_44576822ca.jpg
http://farm2.sta

http://farm4.static.flickr.com/3260/2492729079_2aed94514b.jpg
http://farm4.static.flickr.com/3049/2602123185_254749735f.jpg
http://farm2.static.flickr.com/1322/1448111159_6697cad2a8.jpg
http://farm3.static.flickr.com/2234/2167268318_f97cd11159.jpg
http://farm3.static.flickr.com/2340/2478837754_bd1ff13944.jpg
http://static.flickr.com/87/231698153_b3e27fc211.jpg
http://farm3.static.flickr.com/2035/2359142241_1374c4cab1.jpg
http://farm1.static.flickr.com/46/145308544_5bfb02f5ac.jpg
http://farm3.static.flickr.com/2382/2444440896_e7b1877c3d.jpg
http://farm4.static.flickr.com/3247/2600672325_defc777e1c.jpg
http://farm1.static.flickr.com/252/453920158_06432cbf6c.jpg
http://farm3.static.flickr.com/2190/2493576070_cb14aec71c.jpg
http://farm4.static.flickr.com/3289/2675229236_b5b92e8cab.jpg
http://farm2.static.flickr.com/1150/1350872014_8ed8e8d0de.jpg
http://farm4.static.flickr.com/3253/2732608587_e86002f169.jpg
http://farm4.static.flickr.com/3290/2521011534_fd41f4c824.jpg
http://farm4.static.fl

http://farm4.static.flickr.com/3171/2352879683_c38e45ab6e.jpg
http://farm1.static.flickr.com/184/382955436_d8d62477d3.jpg
http://static.flickr.com/43/121335594_538788d9dc.jpg
http://www.saratogamuseum.org/images/cv-60a2.jpg
http://farm4.static.flickr.com/3039/2640494356_afd6568662.jpg
http://static.flickr.com/98/232539594_03cf11f02f.jpg
http://bridgehands.com/Reviews/Bridge_Cruises/Queen_Elizabeth_II.jpg
http://farm1.static.flickr.com/119/306516984_b712a20ff9.jpg
http://farm1.static.flickr.com/169/445326229_ba409b1f28.jpg
http://farm1.static.flickr.com/47/138502952_cdf0c6262d.jpg
http://farm1.static.flickr.com/39/118622081_b6c0b9ef55.jpg
http://farm1.static.flickr.com/168/437999446_38281c84a0.jpg
http://static.flickr.com/2368/2077116703_be08dcc33b.jpg
http://farm1.static.flickr.com/161/423543687_edfda8acc2.jpg
http://www.vos.noaa.gov/MWL/aug_06/Images/shipwreck.jpg
http://static.flickr.com/2118/2041510746_3a83386b31.jpg
http://www.soest.hawaii.edu/SOEST_News/PressReleases/AGU2003/KM%20

http://farm3.static.flickr.com/2349/2370921915_ddac030406.jpg
http://farm1.static.flickr.com/44/138502844_764fffd900.jpg
http://static.flickr.com/201/524458663_2721a72508.jpg
http://lh3.ggpht.com/_Z4TXFpxoEuU/SGnGvSQsB9I/AAAAAAAAAg0/FzKjUPdPJKw/IMG_6486.JPG
http://farm1.static.flickr.com/237/515550861_39a7a544e8.jpg
http://farm2.static.flickr.com/1153/1342047740_b8639a8c96.jpg
http://farm3.static.flickr.com/2035/2525264755_e9d6194b64.jpg
http://farm1.static.flickr.com/42/120051826_c6cd0bfc42.jpg
http://farm1.static.flickr.com/158/423544592_ccdc979eea.jpg
http://farm1.static.flickr.com/187/468322192_7e4cb119c2.jpg
http://farm1.static.flickr.com/6/5764846_45024d12f6.jpg
http://static.flickr.com/51/137124335_7407a6eca9.jpg
http://www.spacewar.com/images/littoral-combat-ship-freedom-lcs1-bg.jpg
http://farm1.static.flickr.com/165/423543550_51be0c8682.jpg
http://farm3.static.flickr.com/2208/2237664545_d420eaa0b9.jpg
http://farm1.static.flickr.com/139/345479194_6ceee30c16.jpg
http://lh3.ggpht

In [ ]:
url_downalod(bikes_split_urls, 'imagenet/bikes', 'bike')

In [ ]:
train_datagen  = ImageDataGenerator()
test_datagen = ImageDataGenerator()
    
train_generator = train_datagen.flow_from_directory(
        'imagenet/train/',
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'imagenet/validation/',
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')

In [ ]:
model2 =  Sequential()
model2.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3))) 
model2.add(Conv2D(8, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))


model2.add(Flatten())
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(2, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model2.fit_generator(
        train_generator,
        steps_per_epoch=100, # 2000
        epochs=2, validation_data=validation_generator   #65
        )

In [ ]:
img_path = 'imagenet/ship.jpg'
img = load_img(img_path, target_size=(32, 32))
x = img_to_array(img)
print(x.shape)
x = np.expand_dims(x, axis=0)
print(x.shape)
preds = model2.predict(x)

print(preds)
print('Probability that the image is a Bicycle:', preds[0,0])
print('Probability that the image is a Ship:', preds[0,1])

In [ ]:
batch_size = 16

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150,150),
        class_mode = 'binary')
score = model2.evaluate_generator(test_generator, steps=800//16)
print(score[1])